In [24]:
import akantu as aka
import numpy as np
from matplotlib import pyplot as plt
# import DFMesh_aka 
import DFModel_aka
import DFPosprocess_aka
import subprocess

In [25]:
# Material
young_modulus = 275.0*10**9   # (Pa)
density = 2750.0 # (kg/m3)
fracture_energy = 300.  # (N/m)
stress_limit = 300.0   # (Pa)

# Geometry
bar_length = 50 * 10** -3  # (m)
x0 = -0.5 * bar_length  # Left extremitiy x coordinate / 0-initial
xf = 0.5 * bar_length  # Rigth extremitiy x coordinate / f-final
n_elements = 100 # Total number of triangular elements
area = 1.  # Cross sectional area (m2) (Equal to element size )

# Load
strain_rate = 10.0**7  # (s-1)


# Time
time_simulation = 3.0 * 10**-6  # Total time of simulation (s)

In [26]:
# Compute the size of elements (h) for uniform mesh
h_uniform = bar_length / n_elements

geometry_file = f"""
Point(1) = {{ {x0}, 0, 0, {h_uniform} }};
Point(2) = {{ {xf}, 0, 0, {h_uniform} }};
Point(3) = {{ {xf}, {h_uniform}, 0, {h_uniform} }};
Point(4) = {{ {x0}, {h_uniform}, 0, {h_uniform} }};

Line(1) = {{1,2}};
Line(2) = {{2,3}};
Line(3) = {{3,4}};
Line(4) = {{4,1}};

Line Loop(1) = {{1,2,3,4}};
Plane Surface(1) = {{1}};
Physical Surface(1) = {{1}};
Physical Line("Yblocked") = {{1,3}};
Physical Line("right") = {{2}};
Physical Line("left") = {{4}};

Transfinite Surface {1} Left;
"""

subprocess.run("mkdir LOG", shell=True)
subprocess.run("rm -r paraview_backup", shell=True)
subprocess.run("mv paraview paraview_backup", shell=True)

# with open('src_akantu/LOG/bar.geo', 'w') as f:
with open('LOG/bar.geo', 'w') as f:
    f.write(geometry_file)

mkdir: cannot create directory ‘LOG’: File exists


In [27]:
# Read material file
aka.parseInput('../LOG/material.dat')

# Read mesh
spatial_dimension = 2
mesh = aka.Mesh(spatial_dimension)
mesh.read('../LOG/bar.msh')

# Get connectivity list
connect = mesh.getConnectivity(aka._triangle_3)
# Get coordinates 
coords = mesh.getNodes()


# Create model
model = aka.SolidMechanicsModelCohesive(mesh)
model.initFull(_analysis_method=aka._static, _is_extrinsic=True)
# Configure static solver
solver = model.getNonLinearSolver('static')
solver.set('max_iterations', 100)
solver.set('threshold', 1e-10)
solver.set("convergence_type", aka.SolveConvergenceCriteria.residual)
# Add solver 
model.initNewSolver(aka._explicit_lumped_mass)

In [28]:
# Configure the insertion

# Facets mesh and connectivity
mesh_facets = mesh.getMeshFacets()
connect_facets = mesh_facets.getConnectivities()

# Get the cohesive inserter and check facets
cohesive_inserter = model.getElementInserter()
check_facets = cohesive_inserter.getCheckFacets()
up = np.array([0.0, 1.0])
# Loop by all facet types used in the simulation 
for facet_type in connect_facets.elementTypes(dim=(spatial_dimension - 1)):
    conn_facettype = connect_facets(facet_type)
    check_facettype = check_facets(facet_type)
    for el, conn_facettype in enumerate(conn_facettype):
        # Check the direction of the vector 
        dir_vec = coords[conn_facettype[1], :] - coords[conn_facettype[0], :]
        direction = (dir_vec / np.linalg.norm(dir_vec)).dot(up)
        # If the direction is not 1 it means that is a diagonal facet, then assign False
        if abs(direction) < 0.9:
            check_facettype[el] = False


In [29]:
# Set time increment
dt_crit = model.getStableTimeStep()     # Critical time step (s)
dt = dt_crit                        # Adopted time step
model.setTimeStep(dt)
n_steps = int(3 * 10**-3 /dt)       # Number of time steps


# Apply Dirichlet BC to block dispacements at y direction on top and botton of the elements
model.applyBC(aka.FixedValue(0., aka._y), 'Yblocked')



# Set velocity applied at the boundaries
# Value of the velocity at the bounary
vel = strain_rate * bar_length * 0.5
# Apply constant velocity at the boundaries
functor_left = DFModel_aka.FixedVelocity(aka._x, -vel)
functor_right = DFModel_aka.FixedVelocity(aka._x, vel)
model.applyBC(functor_left, 'left')
model.applyBC(functor_right, 'right')


In [30]:
# Initial values
n_nodes = mesh.getNbNodes()     # Number of nodes of the mesh
u0 = model.getDisplacement()    # Initial displacement
v0 = model.getVelocity()        # Initial velocity
# Set initial velocity profile
v0[:,0] = np.array([strain_rate * x for x,y in mesh.getNodes()])
# DFPlot2d.PlotByCoord(v0[:,0],mesh,'initial vel')
# Apply initial velocity values

coh_id = model.getMaterial('insertion').getElementFilter()(aka._cohesive_2d_4)
coh_material = model.getMaterial('insertion')
print(coh_material)
sigmac = coh_material.getInternalReal("sigma_c")
# sigmac = sigmac(aka._cohesive_2d_4)
# print(coh_id)
print(sigmac)


Material cohesive_linear [
 r-p G_c      [Mode I fracture energy]               : 300
 r-p beta     [Beta parameter]                       : 0
 r-p contact_after_breaking [Activation of contact when the elements are fully damaged] : false
 r-p delta_c  [Critical displacement]                : 0
 --p eigen_grad_u [EigenGradU]                       : [[0, 0], [0, 0]]
 r-p finite_deformation [Is finite deformation]      : false
 iii inelastic_deformation [Is inelastic deformation] : false
 r-p kappa    [Kappa parameter]                      : 1
 r-p m_s      [Weibull exponent for sigma_c scaling] : 1
 r-p max_quad_stress_insertion [Insertion of cohesive element when stress is high enough just on one quadrature point] : false
 r-p name                                            : insertion
 r-p penalty  [Penalty coefficient]                  : 3.33805e+14
 rwp recompute [recompute solution]                  : false
 rwp rho      [Density]                              : 0
 r-p sigma_c  [Cri

In [31]:
# VTK plot
# model.setBaseName('bar')
# model.addDumpFieldVector('displacement')
# model.addDumpFieldVector('velocity')
# model.addDumpField('strain')
# model.addDumpField('stress')
# model.addDumpField('blocked_dofs')
# model.addDumpField('material_index')

# # VTK plot for Cohesive model
# model.setBaseNameToDumper('cohesive elements', 'cohesive')
# model.addDumpFieldVectorToDumper('cohesive elements', 'displacement')
# model.addDumpFieldToDumper('cohesive elements', 'damage')
# model.addDumpFieldVectorToDumper('cohesive elements', 'tractions')
# model.addDumpFieldVectorToDumper('cohesive elements', 'opening')

In [32]:
# Initiation of variables
Epot = np.zeros(n_steps)            # Potential energy 
Ekin = np.zeros(n_steps)            # Kinetic energy 
Edis = np.zeros(n_steps)            # Dissipated energy 
Erev = np.zeros(n_steps)            # Reversible energy 
Econ = np.zeros(n_steps)            # Contact energy 
Wext = np.zeros(n_steps)            # External work 
work = 0.0                          # External work from the previous time-step 
fp_left = 0.0                       # Equivalent force applied at the left boundary in the previous time-step
fp_right = 0.0                      # Equivalent force applied at the right boundary in the previous time-step
avg_stress = np.zeros(n_steps)      # Average stress in the whole bar


# Initiation of fragmentatio postprocess variables
nfrag = np.zeros(n_steps)           # Number of fragments  
mean_nelfrag = np.zeros(n_steps)    # Mean number of elements per fragment
sfrag = bar_length                  # Mean size of fragment
avg_sfrag = np.zeros(n_steps)      # Mean fragment size
datahist = []                       # Data of histogram of fragment size
mean_velfrag = np.zeros(n_steps)    # Mean fragments velocity

In [33]:
for n in range(n_steps):

    # Apply velocity at the boundaries 
    functor_left.set_time(dt*n)
    functor_right.set_time(dt*n)
    model.applyBC(functor_left, 'left')
    model.applyBC(functor_right, 'right')

    # VTK files
    # if n%100 == 0:
    #     model.dump()
    #     model.dump('cohesive elements')

    # Run simulation
    model.checkCohesiveStress()
    model.solveStep('explicit_lumped')


    u = model.getDisplacement()[:,0]
    v = model.getVelocity()[:,0]
    acel = model.getAcceleration()[:,0]
    # DFPlot2d.PlotByCoord(v0[:,0],mesh,'v')
    fint = model.getInternalForce()[:,0]
    stress = model.getMaterial(0).getStress(aka._triangle_3)
    stress_xx = stress[:,0]
    avg_stress[n] = np.mean(stress_xx)

    coh_id = model.getMaterial('insertion').getElementFilter()(aka._cohesive_2d_4)
    coh_material = model.getMaterial('insertion')
    # print(coh_material)
    sigmac = coh_material.getInternalReal("sigma_c")
    # sigmac = sigmac(aka._cohesive_2d_4)
    print(coh_id)

    # Energy balance
    Epot[n] = model.getEnergy('potential')
    Ekin[n] = model.getEnergy('kinetic')
    Edis[n] = model.getEnergy('dissipated')
    Erev[n] = model.getEnergy('reversible')
    Econ[n] = model.getEnergy('cohesive contact')
    Wext[n], fp_left, fp_right = DFPosprocess_aka.ExternalWork(mesh, fint, fp_left, fp_right, work, vel, dt)
    work = Wext[n]

    coh = model.getMaterial(1)
    d = model.getMaterial(1).getInternalReal('damage')
    d = d(aka._cohesive_2d_4)
    print(d)
    coh_id = model.getMaterial('insertion').getElementFilter()(aka._cohesive_2d_4)

    # # Fragmentation data
    # fragmentdata = aka.FragmentManager(model)
    # fragmentdata.computeAllData()

    # # Number of fragments
    # nfrag[n] = fragmentdata.getNbFragment()                             # Number of fragments at time-step n   
    # mean_nelfrag[n] = np.mean(fragmentdata.getNbElementsPerFragment())  # Mean number of elements per fragment at time-step n 
    
    # # Fragments size (assuming uniform mesh)
    # sfrag = np.zeros(fragmentdata.getNbFragment())                      
    # sfrag = fragmentdata.getNbElementsPerFragment() * h_uniform      # Sizes of all fragments 
    # avg_sfrag[n] = (mean_nelfrag[n]%2 + (mean_nelfrag[n] - mean_nelfrag[n]%2)/2 ) * h_uniform                      # Mean size of fragments 
    # # datahist = plt.hist(sfrag,10)


    # # Fragments velocities
    # velfrag = np.zeros(fragmentdata.getNbFragment())
    # velfrag = fragmentdata.getVelocity()                # Velocities of all fragments 
    # mean_velfrag[n] = np.mean(velfrag)                  # Mean velocity of fragments
    
    # # Fragments mass
    # mfrag = np.zeros(fragmentdata.getNbFragment())
    # mfrag = fragmentdata.getMass()                      # Fragments mass of all fragments




# # Variation of energy [Energy, time] returns the difference of energy value between time t and t0 
# varEkin, varEpot, varEdis, varErev, varEcon, varWext, varEtot = DFPosprocess_aka.VarEnergy(Epot, Ekin, Edis, Erev, Econ, Wext, n_steps)

# # Power [Energy, time] returns the energy difference between consecutive time steps
# PEkin, PEpot, PEdis, PErev, PEcon, PWext, PEtot = DFPosprocess_aka.Power(Epot, Ekin, Edis, Erev, Econ, Wext, n_steps)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[